In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time

start_time = time.time()

In [2]:
def read_data( train_file):
    with open(train_file) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    print(np.shape(content))
    print(content)
    content = [word for i in range(len(content)) for word in content[i].split()]
    print(content)
    content = np.array(content)
    return content

In [3]:
train_file = "sample.txt"
training_data = read_data(train_file)
print("Loaded training data...")

(1,)
['long ago , the mice had a general council to consider what measures they could take to outwit their common enemy , the cat . some said this , and some said that but at last a young mouse got up and said he had a proposal to make , which he thought would meet the case . you will all agree , said he , that our chief danger consists in the sly and treacherous manner in which the enemy approaches us . now , if we could receive some signal of her approach , we could easily escape from her . i venture , therefore , to propose that a small bell be procured , and attached by a ribbon round the neck of the cat . by this means we should always know when she was about , and could easily retire while she was in the neighbourhood . this proposal met with general applause , until an old mouse got up and said  that is all very well , but who is to bell the cat ? the mice looked at one another and nobody spoke . then the old mouse said it is easy to propose impossible remedies .']
['long', 'ago

In [4]:
#create a dictionary and reverse dictionary and return it
def build_dataset_using_collections( data ):
    counts = collections.Counter(data).most_common()  #using most_common() to get a list
    #print(counts)
    dictionary = dict()
    
    #creating word to id mapping
    for word, _ in counts:
        dictionary[word] = len(dictionary)
    return dictionary
        
def build_dataset( data ):
    counts = list(set( data))
    #print(counts)
    dictionary = dict()
    
    #creating word to id mapping
    for word in counts:
        dictionary[word] = len(dictionary)
        
    reverse_dict = dict((dictionary[k], k) for k in dictionary)
    return dictionary, reverse_dict

In [5]:
#dictionary, reverse_dictionary = build_dataset(training_data)
dictionary,reverse_dict = build_dataset(training_data)
print("Dict using set")
print(dictionary)

"""
#Getting same thing with list so why use collections
dictioanry = build_dataset_using_collections(training_data)
print("Dict using collections")
print(dictionary)
"""

vocab_size = len(dictionary)
print("Vocab size" , vocab_size)

Dict using set
{'all': 0, 'consider': 1, 'retire': 2, 'consists': 3, 'danger': 50, 'had': 6, 'young': 7, 'should': 8, 'to': 9, 'outwit': 10, 'easy': 11, 'take': 100, 'then': 12, 'means': 14, 'very': 15, 'propose': 16, 'know': 17, 'they': 18, 'now': 19, 'enemy': 20, 'always': 21, 'signal': 22, 'she': 36, 'small': 24, 'round': 25, 'old': 26, 'some': 27, 'escape': 28, 'our': 29, 'ribbon': 30, 'what': 31, 'said': 32, 'got': 33, 'approach': 34, '?': 35, 'common': 23, 'be': 37, 'we': 38, 'measures': 103, 'met': 40, 'approaches': 41, 'by': 42, 'about': 44, 'last': 45, 'her': 46, 'receive': 47, 'of': 48, 'could': 49, 'bell': 5, 'one': 4, 'another': 52, 'impossible': 53, 'from': 54, 'spoke': 55, 'would': 56, 'neck': 105, 'long': 57, '.': 58, 'their': 59, 'therefore': 60, 'proposal': 61, 'was': 62, 'until': 63, 'remedies': 64, 'a': 104, 'that': 66, 'treacherous': 67, 'nobody': 68, 'but': 69, 'mice': 70, 'general': 90, 'with': 72, ',': 51, 'he': 73, 'case': 74, 'sly': 75, 'this': 76, 'attached': 

In [6]:

def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"


In [7]:
logs_path = '/tmp/tensorflow/rnn_words'
writer = tf.summary.FileWriter(logs_path)

# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 3

# number of units in RNN cell
n_hidden = 512

weights = tf.Variable(tf.random_normal([n_hidden,vocab_size]))
biases = tf.Variable(tf.random_normal([vocab_size])) 

x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

In [8]:
def RNN( x, weights, biase ):
    
    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights) + biases  

In [9]:
pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [ ]:

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        #print(np.shape(symbols_in_keys)) (3,1)
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        #print(np.shape(symbols_in_keys)) (1,3,1)
        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        #print(np.shape(symbols_out_onehot))  (112,)
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])
        #print(np.shape(symbols_out_onehot))   (1,112)

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dict[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    #print("\ttensorboard --logdir=%s" % (logs_path))
    #print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        print(words)
        if len(words) != n_input:
            continue
        #try:
        symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
        for i in range(32):
            keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
            onehot_pred = session.run(pred, feed_dict={x: keys})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            sentence = "%s %s" % (sentence,reverse_dict[onehot_pred_index])
            symbols_in_keys = symbols_in_keys[1:]
            symbols_in_keys.append(onehot_pred_index)
        print(sentence)
        #except:
         #   print("Word not in dictionary")


Iter= 1000, Average Loss= 4.839817, Average Accuracy= 2.60%
['nobody', 'spoke', '.'] - [then] vs [that]
Iter= 2000, Average Loss= 3.332799, Average Accuracy= 11.20%
['cat', '?', 'the'] - [mice] vs [said]
Iter= 3000, Average Loss= 2.643137, Average Accuracy= 26.00%
['this', 'proposal', 'met'] - [with] vs [.]
Iter= 4000, Average Loss= 2.729301, Average Accuracy= 33.30%
['by', 'this', 'means'] - [we] vs [we]
Iter= 5000, Average Loss= 2.215747, Average Accuracy= 40.70%
['attached', 'by', 'a'] - [ribbon] vs [ribbon]
Iter= 6000, Average Loss= 2.012053, Average Accuracy= 48.20%
[',', 'to', 'propose'] - [that] vs [that]
Iter= 7000, Average Loss= 1.528102, Average Accuracy= 62.30%
[',', 'we', 'could'] - [easily] vs [easily]
Iter= 8000, Average Loss= 1.717492, Average Accuracy= 59.70%
['which', 'the', 'enemy'] - [approaches] vs [approaches]
Iter= 9000, Average Loss= 1.552223, Average Accuracy= 61.60%
['chief', 'danger', 'consists'] - [in] vs [in]
Iter= 10000, Average Loss= 1.407637, Average Accu